In [1]:
import BloomClassify as BC
import LSIsimilarity
from baseline import Baseline
import os
import languageProcess as lp
import matplotlib.pyplot as plt
import numpy as np

In [2]:
LSIs = LSIsimilarity.LSIsimilarity()
CL = BC.BloomClassify(prct = 40, art = 50, baselineFolder="../Baseline/50/plaindata")

def loadModels():
    LSIs.train(basepath="../Baseline/1000/plaindata",noOfTrainArticle=1000)
    CL.load_BL()

def checkArticle(title):
    #title ='Football'
    # check if Validationfolder exists
    if not os.path.exists("../Validation"): os.mkdir("../Validation")

    # get article from wikipedia API
    Base = Baseline(folder="Validation")
    filename = "../Validation/"+title.replace(' ','')+"_raw.txt"
    with open(filename, 'w') as the_file:
        the_file.write(Base.get_dumptext([title]))

    #convert to plain text
    os.system("../wikiextractor/WikiExtractor.py "+"../Validation/"+title.replace(' ','')+"_raw.txt"+" -o"+" ../Validation/"+title.replace(' ','')+"/ --json")

    #housekeeping
    category = os.listdir("../Validation")
    if '.DS_Store' in category: category.remove('.DS_Store')

    #languageProcess
    filepath = "../Validation/"+title.replace(' ','')+"/AA/wiki_00"
    bfarticle = lp.languageProcess(filepath).getHighFreqWords()
    lsiarticle = lp.languageProcess(filepath)
    
    assert (len(bfarticle)>= 1),"No article with that name"
    
    #BloomFilter
    vali_dict = {}
    numOfCheckWords = 100

    for cat in CL.BFdict.keys():
        vali_dict[cat] = sum([1 for word in bfarticle[0].most_common(numOfCheckWords) if CL.BFdict[cat].classify(word[0])])/numOfCheckWords

    #Rewrite to return LSIres and BFres
    return LSIs.compare(lsiarticle.getWords()), vali_dict

def plotGraph(resLSI,resBF):
    ind = np.arange(len(resLSI))    # the x locations for the groups
    width = 0.2         # the width of the bars

    fig, ax = plt.subplots(figsize=(11,5))

    p1 = ax.bar(ind-width/2,list(resLSI.values()),width, align='center',color='dodgerblue')
    p2 = ax.bar(ind+width/2,list(resBF.values()),width, align='center',color='darkorange')

    ax.set_xticks(ind + width/2)
    ax.set_xticklabels(list(resLSI.keys()),rotation='vertical')
    ax.legend((p1[0], p2[0]), ('LSI1000','bloomfilter+tfidf'))
    ax.set_ylim([0,1])
    ax.set_ylabel('right assigned articles')
    ax.grid()
    plt.tight_layout()

    plt.show()

def main():
    loadModels()
    x=0
    while x<1:
        title = input("Insert Article name: ")
        #is title? exception
        print(title)
        resLSI, resBF = checkArticle(title)
        #placeholder resLSI for resBF
        plotGraph(resLSI,resBF)
        x+=1
    pass


In [3]:
main()

loaded Traindata
Insert Article name: Football
Football
[(0, -0.25042945), (1, -0.25546446), (2, -0.24348165), (3, -0.17998138), (4, -0.2131941), (5, -0.2841851), (6, -0.4069546), (7, -0.23161757), (8, -0.26864183), (9, -0.088781655), (10, -0.23297082), (11, -0.3341441), (12, -0.16107105), (13, -0.24239804), (14, -0.20430112), (15, -0.9885759), (16, -0.1867606), (17, -0.18072295), (18, -0.16733406), (19, -0.21451116), (20, -0.19868582), (21, -0.25184923), (22, -0.26710597), (23, -0.06278953), (24, -0.14387292), (25, -0.2588058), (26, -0.31244478)]


IndexError: list index out of range